In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
import numpy as np


In [ ]:
def calculate_and_save_metrics(y_test, y_pred, model_name, label, first = False, last = False, results = None, savename = ''):

    accuracy = round(accuracy_score(y_test, y_pred), 5)
    
    if label == [0,1]:
        precision = round(precision_score(y_test, y_pred, pos_label=1),5)
        recall = round(recall_score(y_test, y_pred, pos_label=1), 5)
        f1 = round(f1_score(y_test, y_pred, pos_label=1), 5)
    else:
        precision = round(precision_score(y_test, y_pred, pos_label='true'),5)
        recall = round(recall_score(y_test, y_pred, pos_label='true'), 5)
        f1 = round(f1_score(y_test, y_pred, pos_label='true'), 5)

    print('Accuracy: ', accuracy)
    print('Precision: ', precision)
    print('Recall: ', recall)
    print('F1: ', f1)

    print(confusion_matrix(y_test, y_pred, labels=label))

    if first:
        results = pd.DataFrame([[accuracy, precision, recall, f1]], columns=['accuracy', 'precision', 'recall', 'f1'], index=[model_name])
    else:
        results.loc[model_name] = [accuracy, precision, recall, f1]

    if last:
        results.to_csv(savename)

    return results



In [3]:

# parent directory

parent_dir = os.path.split(os.getcwd())[0]

#open the csv files
csv_path_fake = os.path.join( parent_dir, 'News_dataset', 'Fake.csv')
csv_path_true = os.path.join( parent_dir, 'News_dataset', 'True.csv')

df_fake = pd.read_csv(csv_path_fake)
df_true = pd.read_csv(csv_path_true)


In [ ]:
#add a label column to the dataframes
df_fake['label'] = 'fake'
df_true['label'] = 'true'

df = pd.concat([df_fake, df_true], ignore_index=True)

# take only the text and label columns
df = df[['text', 'label']]

# drop the nan values
df = df.dropna()

#save the dataframe to a csv file
df.to_csv('news.csv', index=False)

In [8]:
# open the csv file with pandas

df = pd.read_csv('news.csv')

#split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.1, random_state=7)

#initialize a TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.9)

#fit and transform train set, transform test set
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

In [10]:
label = ['fake', 'true']

#initialize a PassiveAggressiveClassifier
pac = PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train, y_train)

#predict on the test set and calculate accuracy
y_pred = pac.predict(tfidf_test)


results = calculate_and_save_metrics(y_test, y_pred, 'PassiveAggressiveClassifier', label, True)

Accuracy:  0.99443
Precision:  0.99532
Recall:  0.993
F1:  0.99416
[[2336   10]
 [  15 2129]]


In [11]:
#with logistic regression
logreg = LogisticRegression()
logreg.fit(tfidf_train, y_train)
y_pred = logreg.predict(tfidf_test)


results = calculate_and_save_metrics(y_test, y_pred, 'LogisticRegression', label, results=results)

Accuracy:  0.98664
Precision:  0.98646
Recall:  0.98554
F1:  0.986
[[2317   29]
 [  31 2113]]


In [12]:
#with multinomial naive bayes

nb = MultinomialNB()
nb.fit(tfidf_train, y_train)
y_pred = nb.predict(tfidf_test)


results = calculate_and_save_metrics(y_test, y_pred, 'MultinomialNB', label=label, results=results)

Accuracy:  0.93964
Precision:  0.94112
Recall:  0.9319
F1:  0.93649
[[2221  125]
 [ 146 1998]]


In [13]:
# with KNN

knn = KNeighborsClassifier()
knn.fit(tfidf_train, y_train)
y_pred = knn.predict(tfidf_test)


results = calculate_and_save_metrics(y_test, y_pred, 'KNeighborsClassifier', label=label, results=results)


Accuracy:  0.65612
Precision:  0.95872
Recall:  0.29244
F1:  0.44818
[[2319   27]
 [1517  627]]


In [14]:
# with a random forest classifier

rf = RandomForestClassifier()
rf.fit(tfidf_train, y_train)
y_pred = rf.predict(tfidf_test)


results = calculate_and_save_metrics(y_test, y_pred, 'RandomForestClassifier', label=label, results=results)

Accuracy:  0.9902
Precision:  0.98883
Recall:  0.99067
F1:  0.98975
[[2322   24]
 [  20 2124]]


In [15]:
# with a support vector machine

svc = LinearSVC(random_state=0, dual='auto')
svc.fit(tfidf_train, y_train)
y_pred = svc.predict(tfidf_test)


results = calculate_and_save_metrics(y_test, y_pred, 'LinearSVC', label=label, results=results, last=True, savename='results_kaggle.csv')

Accuracy:  0.99465
Precision:  0.99579
Recall:  0.993
F1:  0.9944
[[2337    9]
 [  15 2129]]


In [3]:
# using Liar dataset
parent_dir = os.path.split(os.getcwd())[0]

csv_path_liar_train = os.path.join( parent_dir, 'liar_dataset', 'train.tsv')
csv_path_liar_test = os.path.join( parent_dir, 'liar_dataset', 'test.tsv')

df_liar_train = pd.read_csv(csv_path_liar_train, sep='\t', header=None)
df_liar_test = pd.read_csv(csv_path_liar_test, sep='\t', header=None)

df_liar_train.columns = ['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party', 'barely_true', 'false', 'half_true', 'mostly_true', 'pants_on_fire', 'context']
df_liar_test.columns = ['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party', 'barely_true', 'false', 'half_true', 'mostly_true', 'pants_on_fire', 'context']
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.9)

#fit and transform train set, transform test set
tfidf_train = tfidf_vectorizer.fit_transform(df_liar_train['statement'])
tfidf_test = tfidf_vectorizer.transform(df_liar_test['statement'])

label = ['true', 'mostly-true', 'half-true', 'barely-true', 'false', 'pants-fire']

In [ ]:
#initialize a PassiveAggressiveClassifier

pac = PassiveAggressiveClassifier(max_iter=1000)
pac.fit(tfidf_train, df_liar_train['label'])

#predict on the test set and calculate accuracy
y_pred = pac.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'PassiveAggressiveClassifier', label, first=True)

Accuracy:  0.23362273086029992
Precision:  0.23245313772747705
Recall:  0.23362273086029992
F1:  0.2323593773721207
[[43 38 46 20 52  9]
 [50 52 50 37 37 15]
 [50 56 58 40 40 21]
 [33 45 29 38 46 21]
 [36 49 34 34 86 10]
 [13 10 15 15 20 19]]


In [ ]:
# with logistic regression
logreg = LogisticRegression(max_iter=1000)
logreg.fit(tfidf_train, df_liar_train['label'])
y_pred = logreg.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'LogisticRegression', label, results=results)


Accuracy:  0.250197316495659
Precision:  0.24541341568325675
Recall:  0.250197316495659
F1:  0.24157143838446754
[[47 50 46 13 50  2]
 [50 56 65 25 44  1]
 [28 65 84 37 47  4]
 [22 37 53 44 55  1]
 [29 50 57 26 84  3]
 [ 7 14 25 13 31  2]]


In [ ]:
# with multinomial naive bayes

nb = MultinomialNB()
nb.fit(tfidf_train, df_liar_train['label'])
y_pred = nb.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'MultinomialNB', label=label, results=results)

Accuracy:  0.23756906077348067
Precision:  0.2308894522430316
Recall:  0.23756906077348067
F1:  0.20623444528178886
[[ 15  59  79   3  52   0]
 [ 14  65 115   7  40   0]
 [  7  68 126  17  47   0]
 [  8  37  93  18  56   0]
 [ 12  51  99  10  77   0]
 [  2  18  36   7  29   0]]


/home/def/miniconda3/envs/big_data/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# with KNN

knn = KNeighborsClassifier()
knn.fit(tfidf_train, df_liar_train['label'])
y_pred = knn.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'KNeighborsClassifier', label=label, results=results)

Accuracy:  0.21862667719021311
Precision:  0.2148326623861786
Recall:  0.21862667719021311
F1:  0.2133969072351237
[[31 48 39 33 48  9]
 [34 46 56 39 59  7]
 [24 49 64 59 63  6]
 [18 29 41 54 61  9]
 [26 38 52 46 76 11]
 [ 9  6 19 25 27  6]]


In [ ]:
# with a random forest classifier

rf = RandomForestClassifier()
rf.fit(tfidf_train, df_liar_train['label'])
y_pred = rf.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'RandomForestClassifier', label=label, results=results)

Accuracy:  0.2565114443567482
Precision:  0.26151832390741203
Recall:  0.2565114443567482
F1:  0.24337528869577962
[[ 28  61  37  12  68   2]
 [ 32  72  56  20  60   1]
 [ 21  69  81  15  75   4]
 [ 17  35  44  36  80   0]
 [ 18  40  56  26 103   6]
 [  6  14  11  11  45   5]]


In [ ]:
# with a support vector machine

svc = LinearSVC(random_state=0, dual='auto')
svc.fit(tfidf_train, df_liar_train['label'])
y_pred = svc.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'LinearSVC', label=label, results=results, last=True, savename='results_liar.csv')

Accuracy:  0.24546172059984214
Precision:  0.24380461077723303
Recall:  0.24546172059984214
F1:  0.24410137719093702
[[53 44 51 15 38  7]
 [52 49 53 36 42  9]
 [31 68 67 51 38 10]
 [24 31 47 56 44 10]
 [35 47 46 30 76 15]
 [10 12 21 17 22 10]]


In [4]:
# label true and mostly-true as true, and false, barely-true and pants-fire as false

df_liar_train['label'] = df_liar_train['label'].replace(['mostly-true'], 'true')
df_liar_test['label'] = df_liar_test['label'].replace(['mostly-true'], 'true')
df_liar_train['label'] = df_liar_train['label'].replace(['half-true'], 'true')
df_liar_test['label'] = df_liar_test['label'].replace(['half-true'], 'true')
df_liar_train['label'] = df_liar_train['label'].replace(['barely-true'], 'false')
df_liar_test['label'] = df_liar_test['label'].replace(['barely-true'], 'false')
df_liar_train['label'] = df_liar_train['label'].replace(['pants-fire'], 'false')
df_liar_test['label'] = df_liar_test['label'].replace(['pants-fire'], 'false')


#fit and transform train set, transform test set
tfidf_train = tfidf_vectorizer.fit_transform(df_liar_train['statement'])
tfidf_test = tfidf_vectorizer.transform(df_liar_test['statement'])


label = ['false', 'true']

In [9]:
# initialize a PassiveAggressiveClassifier
pac = PassiveAggressiveClassifier(max_iter=1000)
pac.fit(tfidf_train, df_liar_train['label'])

#predict on the test set and calculate accuracy
y_pred = pac.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'PassiveAggressiveClassifier', label, first=True)

Accuracy:  0.5619573796369376
Precision:  0.609353507565337
Recall:  0.6204481792717087
F1:  0.614850798056905
[[269 284]
 [271 443]]


In [10]:
# with logistic regression
logreg = LogisticRegression(max_iter=1000)
logreg.fit(tfidf_train, df_liar_train['label'])
y_pred = logreg.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'LogisticRegression', label, results=results)

Accuracy:  0.6156274664561957
Precision:  0.6318234610917538
Recall:  0.7619047619047619
F1:  0.6907936507936507
[[236 317]
 [170 544]]


In [11]:
# with multinomial naive bayes

nb = MultinomialNB()
nb.fit(tfidf_train, df_liar_train['label'])
y_pred = nb.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'MultinomialNB', label=label, results=results)

Accuracy:  0.601420678768745
Precision:  0.6033630069238378
Recall:  0.8543417366946778
F1:  0.7072463768115941
[[152 401]
 [104 610]]


In [12]:
# with KNN

knn = KNeighborsClassifier()
knn.fit(tfidf_train, df_liar_train['label'])
y_pred = knn.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'KNeighborsClassifier', label=label, results=results)

Accuracy:  0.5706393054459353
Precision:  0.6073232323232324
Recall:  0.6736694677871149
F1:  0.6387782204515272
[[242 311]
 [233 481]]


In [13]:
# with a random forest classifier

rf = RandomForestClassifier()
rf.fit(tfidf_train, df_liar_train['label'])
y_pred = rf.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'RandomForestClassifier', label=label, results=results)

Accuracy:  0.6085240726124704
Precision:  0.6241457858769932
Recall:  0.7675070028011205
F1:  0.6884422110552765
[[223 330]
 [166 548]]


In [14]:
# with a support vector machine

svc = LinearSVC(random_state=0, dual='auto')
svc.fit(tfidf_train, df_liar_train['label'])
y_pred = svc.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'LinearSVC', label=label, results=results, last=True, savename='results_liar_binary.csv')

Accuracy:  0.6022099447513812
Precision:  0.6363636363636364
Recall:  0.6862745098039216
F1:  0.660377358490566
[[273 280]
 [224 490]]


In [17]:
# WELFake dataset

# parent directory

parent_dir = os.path.split(os.getcwd())[0]

csv_path_welfake = os.path.join( parent_dir, 'WELFake', 'WELFake_Dataset.csv')

df_welfake = pd.read_csv(csv_path_welfake)

# drop the rows with np.nan values on text column
df_welfake = df_welfake.dropna(subset=['text'])

X_train, X_test, y_train, y_test = train_test_split(df_welfake['text'], df_welfake['label'], test_size=0.1, random_state=7)


In [18]:
# initialize a TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.9)

# fit and transform train set, transform test set
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

label = [0,1]

In [22]:
# initialize a PassiveAggressiveClassifier
pac = PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train, y_train)

# predict on the test set and calculate accuracy
y_pred = pac.predict(tfidf_test)


results = calculate_and_save_metrics(y_test, y_pred, 'PassiveAggressiveClassifier', label, first=True)

Accuracy:  0.95992
Precision:  0.95307
Recall:  0.96892
F1:  0.96093
[[3367  175]
 [ 114 3554]]


In [23]:
# with logistic regression

logreg = LogisticRegression()
logreg.fit(tfidf_train, y_train)
y_pred = logreg.predict(tfidf_test)


results = calculate_and_save_metrics(y_test, y_pred, 'LogisticRegression', label, results=results)

Accuracy:  0.94397
Precision:  0.93197
Recall:  0.95992
F1:  0.94574
[[3285  257]
 [ 147 3521]]


In [24]:
# with multinomial naive bayes

nb = MultinomialNB()
nb.fit(tfidf_train, y_train)
y_pred = nb.predict(tfidf_test)


results = calculate_and_save_metrics(y_test, y_pred, 'MultinomialNB', label=label, results=results)

Accuracy:  0.86907
Precision:  0.87051
Recall:  0.87241
F1:  0.87146
[[3066  476]
 [ 468 3200]]


In [25]:
#with KNN

knn = KNeighborsClassifier()
knn.fit(tfidf_train, y_train)
y_pred = knn.predict(tfidf_test)


results = calculate_and_save_metrics(y_test, y_pred, 'KNeighborsClassifier', label=label, results=results)

Accuracy:  0.6301
Precision:  0.58095
Recall:  0.97928
F1:  0.72927
[[ 951 2591]
 [  76 3592]]


In [26]:
# with a random forest classifier

rf = RandomForestClassifier()
rf.fit(tfidf_train, y_train)
y_pred = rf.predict(tfidf_test)


results = calculate_and_save_metrics(y_test, y_pred, 'RandomForestClassifier', label=label, results=results)

Accuracy:  0.93329
Precision:  0.91945
Recall:  0.95229
F1:  0.93558
[[3236  306]
 [ 175 3493]]


In [27]:
# with a support vector machine

svc = LinearSVC(random_state=0, dual='auto')
svc.fit(tfidf_train, y_train)
y_pred = svc.predict(tfidf_test)


results = calculate_and_save_metrics(y_test, y_pred, 'LinearSVC', label=label, results=results, last=True, savename='results_welfake.csv')

Accuracy:  0.96117
Precision:  0.95173
Recall:  0.97301
F1:  0.96225
[[3361  181]
 [  99 3569]]


In [ ]:
# df liar using statement speaker and party

csv_path_liar_train = os.path.join( './', 'liar_dataset', 'train.tsv')
csv_path_liar_test = os.path.join( './', 'liar_dataset', 'test.tsv')

df_liar_train = pd.read_csv(csv_path_liar_train, sep='\t', header=None)
df_liar_test = pd.read_csv(csv_path_liar_test, sep='\t', header=None)


df_liar_train.columns = ['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party', 'barely_true', 'false', 'half_true', 'mostly_true', 'pants_on_fire', 'context']
df_liar_test.columns = ['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party', 'barely_true', 'false', 'half_true', 'mostly_true', 'pants_on_fire', 'context']


df_liar_train = df_liar_train[['label', 'statement', 'speaker', 'party']]
df_liar_test = df_liar_test[['label', 'statement', 'speaker', 'party']]
df_liar_train = df_liar_train.dropna()

# vectorize the statement column
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.9)
tfidf_train = tfidf_vectorizer.fit_transform(df_liar_train['statement'])
tfidf_test = tfidf_vectorizer.transform(df_liar_test['statement'])

# vectorize the speaker column
tfidf_train_speaker = tfidf_vectorizer.fit_transform(df_liar_train['speaker'])
tfidf_test_speaker = tfidf_vectorizer.transform(df_liar_test['speaker'])

# vectorize the party column
tfidf_train_party = tfidf_vectorizer.fit_transform(df_liar_train['party'])
tfidf_test_party = tfidf_vectorizer.transform(df_liar_test['party'])

# concatenate the three vectors
tfidf_train = np.concatenate((tfidf_train.toarray(), tfidf_train_speaker.toarray(), tfidf_train_party.toarray()), axis=1)
tfidf_test = np.concatenate((tfidf_test.toarray(), tfidf_test_speaker.toarray(), tfidf_test_party.toarray()), axis=1)

label = ['true', 'mostly-true', 'half-true', 'barely-true', 'false', 'pants-fire']

In [ ]:
# initialize a PassiveAggressiveClassifier
pac = PassiveAggressiveClassifier(max_iter=1000)
pac.fit(tfidf_train, df_liar_train['label'])

# predict on the test set and calculate accuracy
y_pred = pac.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'PassiveAggressiveClassifier', label, first=True)

Accuracy:  0.22888713496448304
Precision:  0.23023408526590852
Recall:  0.22888713496448304
F1:  0.22897047161020762
[[43 50 48 18 40  9]
 [52 52 63 32 38  4]
 [56 56 66 33 37 17]
 [29 36 47 41 41 18]
 [42 41 50 36 64 16]
 [ 8  8 16 12 24 24]]


In [ ]:
# with logistic regression

logreg = LogisticRegression(max_iter=1000)
logreg.fit(tfidf_train, df_liar_train['label'])
y_pred = logreg.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'LogisticRegression', label, results=results)

Accuracy:  0.2557221783741121
Precision:  0.2622683610451152
Recall:  0.2557221783741121
F1:  0.2548912547110119
[[60 50 43 15 37  3]
 [53 64 70 21 31  2]
 [40 67 68 46 43  1]
 [25 32 63 35 51  6]
 [43 37 48 32 77 12]
 [ 5  9 14 13 31 20]]


In [ ]:
# with multinomial naive bayes

nb = MultinomialNB()
nb.fit(tfidf_train, df_liar_train['label'])
y_pred = nb.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'MultinomialNB', label=label, results=results)

Accuracy:  0.2549329123914759
Precision:  0.26958362898191
Recall:  0.2549329123914759
F1:  0.24034321415470822
[[ 26  70  60   8  42   2]
 [ 17  81 100  10  32   1]
 [ 19  67 102  28  49   0]
 [  9  40  79  20  60   4]
 [ 19  45  80  19  81   5]
 [  2  12  18  11  36  13]]


In [ ]:
# with KNN

knn = KNeighborsClassifier()
knn.fit(tfidf_train, df_liar_train['label'])
y_pred = knn.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'KNeighborsClassifier', label=label, results=results)

Accuracy:  0.22257300710339384
Precision:  0.2314443848997401
Recall:  0.22257300710339384
F1:  0.22221364309019398
[[39 54 38 42 32  3]
 [31 50 66 50 43  1]
 [32 53 68 62 46  4]
 [12 30 35 61 64 10]
 [27 36 46 78 49 13]
 [ 3 10 13 22 29 15]]


In [ ]:
# with a random forest classifier

rf = RandomForestClassifier()
rf.fit(tfidf_train, df_liar_train['label'])
y_pred = rf.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'RandomForestClassifier', label=label, results=results)

Accuracy:  0.2462509865824783
Precision:  0.25598791463936366
Recall:  0.2462509865824783
F1:  0.23947975224915122
[[34 68 43  9 50  4]
 [36 70 58 16 59  2]
 [31 83 63 23 64  1]
 [15 50 45 31 67  4]
 [23 42 52 23 98 11]
 [ 6  9 25  9 27 16]]


In [ ]:
# with a support vector machine

svc = LinearSVC(random_state=0, dual='auto')
svc.fit(tfidf_train, df_liar_train['label'])
y_pred = svc.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'LinearSVC', label=label, results=results, last=True, savename='results_liar_speaker_party.csv')

Accuracy:  0.23993685872138912
Precision:  0.24102368113167535
Recall:  0.23993685872138912
F1:  0.2402004306580156
[[49 52 43 21 37  6]
 [56 57 58 34 30  6]
 [42 62 63 51 34 13]
 [31 33 49 41 42 16]
 [38 37 40 44 74 16]
 [ 7 17 15 12 21 20]]


In [ ]:
# binary liar using statement speaker and party

df_liar_train['label'] = df_liar_train['label'].replace(['mostly-true'], 'true')
df_liar_test['label'] = df_liar_test['label'].replace(['mostly-true'], 'true')
df_liar_train['label'] = df_liar_train['label'].replace(['barely-true'], 'false')
df_liar_test['label'] = df_liar_test['label'].replace(['barely-true'], 'false')
df_liar_train['label'] = df_liar_train['label'].replace(['pants-fire'], 'false')
df_liar_test['label'] = df_liar_test['label'].replace(['pants-fire'], 'false')

# vectorize the statement column
tfidf_train = tfidf_vectorizer.fit_transform(df_liar_train['statement'])
tfidf_test = tfidf_vectorizer.transform(df_liar_test['statement'])

# vectorize the speaker column
tfidf_train_speaker = tfidf_vectorizer.fit_transform(df_liar_train['speaker'])
tfidf_test_speaker = tfidf_vectorizer.transform(df_liar_test['speaker'])

# vectorize the party column
tfidf_train_party = tfidf_vectorizer.fit_transform(df_liar_train['party'])
tfidf_test_party = tfidf_vectorizer.transform(df_liar_test['party'])

# concatenate the three vectors
tfidf_train = np.concatenate((tfidf_train.toarray(), tfidf_train_speaker.toarray(), tfidf_train_party.toarray()), axis=1)
tfidf_test = np.concatenate((tfidf_test.toarray(), tfidf_test_speaker.toarray(), tfidf_test_party.toarray()), axis=1)

label = ['true', 'false']

In [ ]:
# initialize a PassiveAggressiveClassifier

pac = PassiveAggressiveClassifier(max_iter=1000)
pac.fit(tfidf_train, df_liar_train['label'])

# predict on the test set and calculate accuracy
y_pred = pac.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'PassiveAggressiveClassifier', label, first=True)

Accuracy:  0.4262036306235201
Precision:  0.43373585872217285
Recall:  0.4262036306235201
F1:  0.4295301446861111
[[195 142]
 [161 280]]


In [ ]:
# with logistic regression

logreg = LogisticRegression(max_iter=1000)
logreg.fit(tfidf_train, df_liar_train['label'])
y_pred = logreg.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'LogisticRegression', label, results=results)

Accuracy:  0.49171270718232046
Precision:  0.4555053197588091
Recall:  0.49171270718232046
F1:  0.4641482017531922
[[238 171]
 [155 361]]


In [ ]:
# with multinomial naive bayes

nb = MultinomialNB()
nb.fit(tfidf_train, df_liar_train['label'])
y_pred = nb.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'MultinomialNB', label=label, results=results)

Accuracy:  0.5193370165745856
Precision:  0.4799922721754087
Recall:  0.5193370165745856
F1:  0.46245516911100526
[[249 196]
 [144 405]]


In [ ]:
# with KNN

knn = KNeighborsClassifier()
knn.fit(tfidf_train, df_liar_train['label'])
y_pred = knn.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'KNeighborsClassifier', label=label, results=results)

Accuracy:  0.48697711128650356
Precision:  0.46545565923182264
Recall:  0.48697711128650356
F1:  0.47018379312967296
[[188 187]
 [112 381]]


In [ ]:
# with a random forest classifier

rf = RandomForestClassifier()
rf.fit(tfidf_train, df_liar_train['label'])
y_pred = rf.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'RandomForestClassifier', label=label, results=results)

Accuracy:  0.5035516969218626
Precision:  0.4672927839945014
Recall:  0.5035516969218626
F1:  0.44938154763518495
[[236 207]
 [152 397]]


In [ ]:
# with a support vector machine

svc = LinearSVC(random_state=0, dual='auto')
svc.fit(tfidf_train, df_liar_train['label'])
y_pred = svc.predict(tfidf_test)


results = calculate_and_save_metrics(df_liar_test['label'], y_pred, 'LinearSVC', label=label, results=results, last=True, savename='results_liar_speaker_party_binary.csv')

Accuracy:  0.4585635359116022
Precision:  0.4426506927816209
Recall:  0.4585635359116022
F1:  0.4486165963853383
[[223 156]
 [165 316]]
